## AgentCore 런타임(Runtime)에서 동적 클라이언트 등록

이 예제는 동적 클라이언트 등록을 지원하는 AgentCore 런타임(Runtime)에 MCP를 배포(Deployment)하는 방법을 보여줍니다.

이 튜토리얼에서는 이 AgentCore 기능을 지원하는 Auth0와 통합된 예제를 만들 것입니다.

이는 예를 들어 사용자가 소셜 미디어 로그인을 사용하여 MCP에 연결할 수 있도록 동적으로 사용자를 등록하는 데 유용할 수 있습니다.

이 튜토리얼에서 배울 내용:

* 도구(Tool)가 있는 MCP 서버를 만드는 방법
* 서버를 로컬에서 테스트하는 방법
* DCR을 지원하고 API와 앱을 추가하도록 Auth0 테넌트를 구성(Configuration)하는 방법
* Auth0의 DCR과 통합하여 AWS에 서버를 배포(Deployment)하는 방법
* 배포(Deployment)된 서버를 호출(Invocation)하는 방법

### 튜토리얼 세부사항

| 정보                | 세부사항                                                  |
|:--------------------|:---------------------------------------------------------|
| 튜토리얼 유형       | Auth0에서 도구(Tool) 호스팅 + DCR                         |
| 도구(Tool) 유형     | MCP 서버                                                  |
| 튜토리얼 구성 요소   | AgentCore 런타임(Runtime)에 도구(Tool) 호스팅, MCP 서버 생성 |
| 튜토리얼 분야       | 범용                                                      |
| 예제 복잡도         | 중급                                                      |
| 사용 SDK            | Amazon BedrockAgentCore Python SDK 및 MCP Client         |

### 튜토리얼 아키텍처

이 튜토리얼에서는 이 예제를 AgentCore 런타임(Runtime)에 배포(Deployment)하는 방법을 설명합니다.

데모 목적으로 `add_numbers`, `multiply_numbers`, `greet_users` 3가지 도구(Tool)가 있는 매우 간단한 MCP 서버를 사용합니다.

![Architecture](images/architecture.png)

### 튜토리얼 주요 기능

* MCP 서버 호스팅
* 동적 클라이언트 등록(DCR)
* Auth0

In [ ]:
!pip install -Uq -r requirements.txt

**Restart your kernel to reflect installed dependencies.**

**설치된 종속성을 반영하기 위해 커널을 다시 시작하세요.**

## Auth0 구성(Configuration)

### 일반 설정

이 예제에서는 Auth0와 함께 DCR(동적 클라이언트 등록)을 구현합니다. 시작하기 전에 Auth0 콘솔에서 DCR 옵션을 활성화합니다.

- 왼쪽 패널 메뉴에서 `Settings`를 클릭합니다. 그런 다음 **Tenant Settings** 아래에서 `Advanced` 옵션을 클릭합니다. *Dynamic Client Registration (DCR)* 옵션을 찾을 때까지 아래로 스크롤하고 클릭하여 *활성화*합니다. 또한 *Enable Application Connections*를 클릭하여 새 애플리케이션이 생성될 때 연결이 활성화되도록 합니다:

![Enable DCR](images/01_enable_dcr.png)

자세한 내용은 Auth0 [동적 클라이언트 등록](https://auth0.com/docs/get-started/applications/dynamic-client-registration) 문서를 참조하세요.

이제 활성화되었으니 MCP에서 사용자(앱)의 로그인 메커니즘으로 *Google / Gmail* ID를 사용해 보겠습니다.

- 왼쪽 패널 메뉴에서 `Authentication`을 클릭한 다음 `Social`을 클릭합니다. **Social Connections** 화면에서 `google-oauth2` 옵션을 클릭합니다. 설정 화면에서 `Promote Connection to Domain Level` 옵션을 찾을 때까지 아래로 스크롤하고 활성화합니다:

![Enable Social](images/02_enable_social.png)

이제 Auth0 테넌트에 대한 모든 설정을 구성(Configuration)했습니다. 계속해서 Auth0 API를 만들어 보겠습니다.

---

### API 생성

API를 생성해 보겠습니다. 왼쪽 패널 메뉴의 Applications 섹션에서 `APIs`를 클릭합니다. APIs 화면에서 오른쪽 상단 버튼 `+ Create API`를 클릭합니다.

![API](images/03_create_api.png)

- Name: API의 이름.
- Identifier: API의 고유 식별자. 이 값은 인가(Authorization) 호출에서 `audience` 매개변수로 사용됩니다.
- 다른 옵션은 기본값으로 유지하고 Create를 클릭할 수 있습니다.

API 설정에서 `Permissions` 탭을 클릭한 다음 이 패턴 `<identifier_name>:Invoke`에 따라 invoke 권한을 추가합니다:

![Permissions](images/04_permissions.png)

완료. 이제 API가 구성(Configuration)되었습니다. 앱을 만들어 보겠습니다.

---

### 애플리케이션 생성

애플리케이션을 만들어 구성(Configuration)을 완료해 보겠습니다. `Applications` 왼쪽 패널 메뉴에서 `Applications`를 클릭한 다음 `+ Create Application`을 클릭합니다.

앱 생성 화면에서 앱 이름을 지정하고 `Machine-to-Machine` 옵션을 선택한 다음 create를 클릭합니다:

![App](images/05_app.png)

그런 다음 다음 화면에서 API를 선택하라는 메시지가 표시됩니다. 이전 단계에서 만든 API를 선택하고 `all` 버튼을 클릭하여 모든 작업을 인가(Authorization)한 다음 `Authorize` 버튼을 클릭합니다:

![Auth](images/06_authorizing.png)

이제 앱이 생성되었습니다.

---

### 앱 정보 가져오기

이제 예제에서 사용할 수 있도록 앱 정보를 가져와야 합니다.

앱을 클릭하고 `quickstart` 페이지로 이동합니다. 이 페이지에는 필요한 모든 매개변수가 포함된 Linux `curl` 예제가 표시됩니다. `--url` 값에서 도메인을 복사합니다. 다음 예제와 같이:

`--url` 매개변수:

```bash
--url https://<your-auth0-tenant>.us.auth0.com/oauth/token
```

이 노트북에서 사용할 다음 환경 변수를 빌드합니다:

```bash
export DISCOVERY_URL="https://<your-auth0-tenant>.us.auth0.com/.well-known/openid-configuration"
```

In [ ]:
DISCOVERY_URL="https://<your-auth0-tenant>.us.auth0.com/.well-known/openid-configuration"

**여기에 도메인을 교체하세요**

In [ ]:
# Replace if you created with a different identifier
AUDIENCE="ac-runtime-api"

### Creating MCP Server

Now let's create our MCP server with three simple tools:

In [ ]:
%%writefile server.py
from mcp.server.fastmcp import FastMCP
from starlette.responses import JSONResponse

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together"""
    return a + b

@mcp.tool()
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers together"""
    return a * b

@mcp.tool()
def greet_user(name: str) -> str:
    """Greet a user by name"""
    return f"Hello, {name}! Nice to meet you."

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

audience도 입력해야 합니다. 이것은 API 식별자가 됩니다. 이 예제를 따라 `ac-runtime-api`를 사용하면 그것이 audience 값이 됩니다:

In [ ]:
%%writefile test_my_mcp_client.py
import asyncio
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    mcp_url = "http://localhost:8000/mcp"
    headers = {}

    async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
        read_stream,
        write_stream,
        _,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tool_result = await session.list_tools()
            print("Available tools:")
            for tool in tool_result.tools:
                print(f"  - {tool.name}: {tool.description}")

if __name__ == "__main__":
    asyncio.run(main())

To test your MCP server locally:

1. **Terminal 1**: Start the MCP server

```bash
python server.py
```

2. **Terminal 2**: Run the test script

```bash
python test_my_mcp_client.py
```

### MCP 서버 생성

이제 세 가지 간단한 도구(Tool)로 MCP 서버를 생성해 보겠습니다:

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "mcp_dcr_sample"

auth_config = {
    "customJWTAuthorizer": {
        "allowedAudience": [
            AUDIENCE
        ],
        "discoveryUrl": DISCOVERY_URL,
    }
}

response = agentcore_runtime.configure(
    entrypoint="server.py",
    auto_create_execution_role=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
    authorizer_configuration=auth_config,
    protocol="MCP",
    memory_mode="NO_MEMORY",
    deployment_type="direct_code_deploy",
    runtime_type="PYTHON_3_13",
)

In [ ]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()
print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

### Testing

The file `mcp_auth0_client.py` contains a local implementation to connect to our deployed agent on AgentCore Runtime.

When you run your script, it will redirect you to a login page that will ask for your Google identity to log in.

After you log in, it will ask you to authorize Auth0 to create an app using your email.

![Redirect](images/07_redirect.png)

If you accept it, it will redirect to a plain HTML page (implemented in `mcp_auth0_client.py`) showing that it was successful.

![OK](images/08_success.png)

### MCP 서버 로컬 테스트 (선택사항)

원한다면 새로 만든 MCP 서버를 로컬에서 테스트할 수 있습니다.

테스트 Python 스크립트를 생성하려면 다음 셀을 실행합니다.

In [ ]:
agent_arn = launch_result.agent_arn
region = "us-west-2"
custom_endpoint =f"https://bedrock-agentcore.{region}.amazonaws.com"

agent_arn, custom_endpoint, AUDIENCE

In [ ]:
import mcp_auth0_client as mcp_client

await mcp_client.main(agent_arn, custom_endpoint, AUDIENCE)

## Clean UP

In [ ]:
agentcore_runtime.destroy()

MCP 서버를 로컬에서 테스트하려면:

1. **터미널 1**: MCP 서버 시작

```bash
python server.py
```

2. **터미널 2**: 테스트 스크립트 실행

```bash
python test_my_mcp_client.py
```